In [1]:
import platform
print(platform.python_version())

3.8.18


In [3]:
# ## Dependencies
# We will use the [requests](http://docs.python-requests.org/en/master/)
# library, although similar operations are available through the
# [Standard Python Libraries](https://docs.python.org/2/library/internet.html)

import requests
import json
import getpass

In [4]:
# The current URL hosting the ESPA interfaces has reached a stable version 1.0
host = 'https://espa.cr.usgs.gov/api/v1/'

In [5]:
# ESPA uses the ERS credentials for identifying users

username = 'zk2086'
password = getpass.getpass()

 ········


In [6]:
# ## espa_api: A Function
# First and foremost, define a simple function for interacting with the API. 
# 
# The key things to watch for:
# 
# * Always scrub for a `"messages"` field returned in the response, it is only informational about a request
#   * **Errors** (`"errors"`): Brief exlaination about why a request failed
#   * **Warnings** (`"warnings"`): Cautions about a successful response
# * Always make sure the requested HTTP `status_code` returned is valid 
#   * **GET**: `200 OK`: The requested resource was successfully fetched (result can still be empty)
#   * **POST**: `201 Created`: The requested resource was created
#   * **PUT**: `202 Accepted`: The requested resource was updated


def espa_api(endpoint, verb='get', body=None, uauth=None):
    """ Suggested simple way to interact with the ESPA JSON REST API """
    auth_tup = uauth if uauth else (username, password)
    response = getattr(requests, verb)(host + endpoint, auth=auth_tup, json=body)
    print('{} {}'.format(response.status_code, response.reason))
    data = response.json()
    if isinstance(data, dict):
        messages = data.pop("messages", None)  
        if messages:
            print((json.dumps(messages, indent=4)))
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None
    else:
        return data

In [7]:
# ## General Interactions: Authentication
# Basic call to get the current user's information. It requires valid credentials, and is a good check that the system is available

print('GET /api/v1/user')
resp = espa_api('user')
print((json.dumps(resp, indent=4)))

GET /api/v1/user
200 OK
{
    "email": "ZK2086@NYU.EDU",
    "first_name": "ZIYU",
    "last_name": "KONG",
    "roles": [
        "active"
    ],
    "username": "zk2086"
}


In [11]:
# ## General Interactions: Available Options
# 
# ESPA offers several services, descriptions can be found here: 
# * [AVAILABLE-PRODUCTS](/docs/available-products.md)
# * [CUSTOMIZATION](/docs/customization.md)

# Call to demonstrate what is returned from available-products
print('GET /api/v1/available-products')
avail_list = {'inputs': [ 
                         'LC08_L2SP_035032_20160104_20200907_02_T1'
                        ]
             }
resp = espa_api('available-products', body=avail_list)
print(json.dumps(resp, indent=4))

GET /api/v1/available-products
200 OK
{
    "olitirs8_collection_2_l2": {
        "products": [
            "l1",
            "sr_ndvi",
            "sr_evi",
            "sr_savi",
            "sr_msavi",
            "sr_ndmi",
            "sr_nbr",
            "sr_nbr2",
            "sr_ndsi",
            "et"
        ],
        "inputs": [
            "LC08_L2SP_035032_20160104_20200907_02_T1"
        ]
    }
}


In [12]:
# ESPA can produce outputs all of the same geographic projections.  
# 
# Call to show the available projection parameters that can be used:

print('GET /api/v1/projections')
projs = espa_api('projections')
print(json.dumps(list(projs.keys())))

GET /api/v1/projections
200 OK
["aea", "utm", "lonlat", "sinu", "ps"]


In [13]:
# This is a Schema Definition, useful for building a valid order
# 
# Example (*UTM Projection*):

print(json.dumps(projs['utm']['properties'], indent=4))

{
    "zone": {
        "type": "integer",
        "title": "UTM Grid Zone Number",
        "display_rank": 0,
        "required": true,
        "minimum": 1,
        "maximum": 60
    },
    "zone_ns": {
        "type": "string",
        "title": "UTM Hemisphere",
        "display_rank": 1,
        "required": true,
        "enum": [
            "north",
            "south"
        ]
    }
}


In [27]:
# ## Practical Example: Building An Order
# Here we use two different Landsat sensors to build up an order, and then place the order into the system

l8_ls = ['LC08_L1TP_029030_20161109_20170219_01_T1',
         'LC08_L1TP_029030_20160821_20170222_01_T1',
         'LC08_L1TP_029030_20130712_20170309_01_T1']
l7_ls =['LE07_L1TP_029030_20170221_20170319_01_T1',
        'LE07_L1TP_029030_20161101_20161127_01_T1',
        'LE07_L1TP_029030_20130602_20160908_01_T1']

# Differing products across the sensors
l7_prods = ['toa', 'bt']
l8_prods = ['sr']

# Standard Albers CONUS
projection = {'aea': {'standard_parallel_1': 29.5,
                      'standard_parallel_2': 45.5,
                      'central_meridian': -96.0,
                      'latitude_of_origin': 23.0,
                      'false_easting': 0,
                      'false_northing': 0,
                      'datum': 'nad83'}}

# Let available-products place the acquisitions under their respective sensors
# ls = l8_ls + l7_ls

ls = ['LC08_L2SP_035032_20160104_20200907_02_T1']

print('GET /api/v1/available-products')
order = espa_api('available-products', body=dict(inputs=ls))
print((json.dumps(order, indent=4)))


# **NOTE**: Here we will not need to know what the sensor names were for the Product IDs, thanks to the response from this `available-products` resource. 

GET /api/v1/available-products
200 OK
{
    "olitirs8_collection_2_l2": {
        "products": [
            "l1",
            "sr_ndvi",
            "sr_evi",
            "sr_savi",
            "sr_msavi",
            "sr_ndmi",
            "sr_nbr",
            "sr_nbr2",
            "sr_ndsi",
            "et"
        ],
        "inputs": [
            "LC08_L2SP_035032_20160104_20200907_02_T1"
        ]
    }
}


In [28]:
order['olitirs8_collection_2_l2']['products'] = ['sr_ndvi', 'et']

order

{'olitirs8_collection_2_l2': {'products': ['sr_ndvi', 'et'],
  'inputs': ['LC08_L2SP_035032_20160104_20200907_02_T1']}}

In [29]:
# Replace the available products that was returned with what we want
# for sensor in order.keys():
#     if isinstance(order[sensor], dict) and order[sensor].get('inputs'):
#         if set(l7_ls) & set(order[sensor]['inputs']):
#             order[sensor]['products'] = l7_prods
#         if set(l8_ls) & set(order[sensor]['inputs']):
#             order[sensor]['products'] = l8_prods

# Add in the rest of the order information
order['projection'] = projection
order['format'] = 'gtiff'
order['resampling_method'] = 'cc'
order['note'] = 'API Demo Jupyter!!'

# Notice how it has changed from the original call available-products
print((json.dumps(order, indent=4)))

{
    "olitirs8_collection_2_l2": {
        "products": [
            "sr_ndvi",
            "et"
        ],
        "inputs": [
            "LC08_L2SP_035032_20160104_20200907_02_T1"
        ]
    },
    "projection": {
        "aea": {
            "standard_parallel_1": 29.5,
            "standard_parallel_2": 45.5,
            "central_meridian": -96.0,
            "latitude_of_origin": 23.0,
            "false_easting": 0,
            "false_northing": 0,
            "datum": "nad83"
        }
    },
    "format": "gtiff",
    "resampling_method": "cc",
    "note": "API Demo Jupyter!!"
}


In [30]:
# #### Place the order
# Place the order
print('POST /api/v1/order')
resp = espa_api('order', verb='post', body=order)
print((json.dumps(resp, indent=4)))

POST /api/v1/order
201 CREATED
{
    "orderid": "espa-ZK2086@NYU.EDU-03292024-092726-279",
    "status": "ordered"
}


In [31]:
# If successful, we will get our order-id
orderid = resp['orderid']

# ## Check the status of an order
# 

print(('GET /api/v1/order-status/{}'.format(orderid)))
resp = espa_api('order-status/{}'.format(orderid))
print((json.dumps(resp, indent=4)))

GET /api/v1/order-status/espa-ZK2086@NYU.EDU-03292024-092726-279
200 OK
{
    "orderid": "espa-ZK2086@NYU.EDU-03292024-092726-279",
    "status": "ordered"
}


In [32]:
# Now, we can check for any completed products, and get the download url's for completed scenes

print(('GET /api/v1/item-status/{0}'.format(orderid)))
resp = espa_api('item-status/{0}'.format(orderid), body={'status': 'complete'})
print((json.dumps(resp[orderid], indent=4)))

GET /api/v1/item-status/espa-ZK2086@NYU.EDU-03292024-092726-279
200 OK
[]


In [33]:
# Once the order is completed or partially completed, can get the download url's
for item in resp[orderid]:
    print(("URL: {0}".format(item.get('product_dload_url'))))

In [34]:
# # Find previous orders 
# 
# List backlog orders for the authenticated user.
print('GET /api/v1/list-orders')
filters = {"status": ["complete", "ordered"]}  # Here, we ignore any purged orders
resp = espa_api('list-orders', body=filters)
print((json.dumps(resp, indent=4)))


GET /api/v1/list-orders
200 OK
[
    "espa-ZK2086@NYU.EDU-03292024-092726-279",
    "espa-ZK2086@NYU.EDU-03152024-081241-661"
]


In [35]:
# ## Emergency halt an Order
# ### PLEASE BE CAREFUL!
# 
# ESPA processes your orders in the sequence in which they are recieved.  
# You may want to remove blocking orders in your queue, to prioritize your latest orders

# In-process orders
print('GET /api/v1/list-orders')
filters = {"status": ["ordered"]}
orders = espa_api('list-orders', body=filters)

# Here we cancel an incomplete order
orderid = orders[0]
cancel_request = {"orderid": orderid, "status": "cancelled"}
print('PUT /api/v1/order')
order_status = espa_api('order', verb='put', body=cancel_request)

print((json.dumps(order_status, indent=4)))


# # Python Script
# 
# This notebook is available as a script for [download here](/examples/api_demo.py).

GET /api/v1/list-orders
200 OK
PUT /api/v1/order
202 ACCEPTED
{
    "orderid": "espa-ZK2086@NYU.EDU-03292024-092726-279",
    "status": "cancelled"
}
